## Advanced Prompting Patterns

Welcome to the hands-on lab for **Module 3: Advanced Prompting Patterns**. 

In this notebook, we'll walk through practical examples of:  
2. **Chain of Thought (CoT)** prompting  
3. **Tree of Thoughts (ToT)**  
4. **Algorithm of Thoughts**  
5. **Generated Knowledge** prompting  
6. **Rephrase and Respond (RaR)**  



### 1. Setup

Let's define a helper function to send chat prompts to the model for convenience. This function will call the `/v1/chat/completions` endpoint with the given messages and parameters:


In [ ]:
import requests
import json

# Replace with the correct endpoint for your local LLM server
LLM_API_URL = "http://localhost:1234/v1/chat/completions"

# A helper function to send chat-based requests to the locally hosted openchat-3.5-0106 model
def generate_response(
    messages, 
    temperature=0.7, 
    max_tokens=256, 
    top_p=1.0, 
    n=1, 
    stream=False
):
    payload = {
        "model": "openchat-3.5-0106",
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "top_p": top_p,
        "n": n,
        "stream": stream
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(LLM_API_URL, headers=headers, json=payload)
    
    # The structure of the response may vary by implementation.
    # Adjust parsing logic if needed.
    if response.status_code == 200:
        data = response.json()
        # For a ChatCompletion, responses are typically found in data["choices"][i]["message"]["content"]
        # We'll return the first choice's content for simplicity.
        return data["choices"][0]["message"]["content"].strip()
    else:
        raise Exception(f"Request failed with status code {response.status_code} and message: {response.text}")


### 2. Chain of Thought (CoT) Prompting

Chain of Thought prompting encourages the model to reason step-by-step. One common pattern is to explicitly instruct the model to show or use its reasoning (though for production systems, you might keep the chain-of-thought hidden).

#### Example


In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant. "
            "Use detailed reasoning to arrive at the correct answer, but provide a concise final answer."
        )
    },
    {
        "role": "user",
        "content": (
            "A coffee shop sells a latte for $4.00. If they sell 150 lattes in a day, "
            "how much revenue do they make from lattes?"
        )
    },
    {
        "role": "assistant",
        "content": (
            # We request the chain-of-thought reasoning.
            "Let's break down the problem step by step. Then provide a short final answer at the end."
        )
    }
]

response = generate_response(messages, temperature=0.5)
print("Chain-of-Thought Response:\n", response)


**Exercise:**  
- Try adjusting the `messages` to explicitly ask for “show your reasoning” vs. “think through the reasoning but do NOT show it.” Observe the difference in the model’s behavior.
- Experiment with `temperature` to see how the detail or style changes.

### 3. Tree of Thoughts (ToT)

Tree of Thoughts prompting is an approach that encourages the model to branch out multiple possible reasoning paths before converging on a final solution.

**Note:** Implementing ToT often involves multiple calls to the model, manually orchestrating the "branching" and "exploration." Below is a simplified version where we encourage the model to outline multiple solution paths in one call.


In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a problem-solving assistant. "
            "When given a question, explore multiple possible solution paths (like branching thoughts), "
            "evaluate them, and then converge to the best answer."
        )
    },
    {
        "role": "user",
        "content": (
            "I want to invest in the stock market. Outline a few different strategies I could use, "
            "then decide which strategy might work best for a moderate risk profile."
        )
    }
]

response = generate_response(messages, temperature=0.7, max_tokens=400)
print("Tree of Thoughts Response:\n", response)

**Exercise:**  
- Modify the user prompt to be more specific (e.g., “for technology stocks” or “in a bullish market”).  
- Increase the `max_tokens` if the model gets cut off while enumerating multiple paths.

### 4. Algorithm of Thoughts

Algorithm of Thoughts is a prompting technique where you ask the model to apply a systematic, step-by-step procedure to solve a problem. It’s similar to CoT but emphasizes a strict algorithmic approach.

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are an assistant that solves problems using a very systematic approach. "
            "Always outline a clear algorithm or checklist before arriving at a conclusion."
        )
    },
    {
        "role": "user",
        "content": (
            "Determine if the string 'racecar' is a palindrome using a step-by-step checklist."
        )
    }
]

response = generate_response(messages, temperature=0.3)
print("Algorithm of Thoughts Response:\n", response)

**Exercise:**  
- Adapt this approach to a real-world scenario: “Find prime factors of 84 using a systematic approach.”

### 5. Generated Knowledge Prompting

Generated Knowledge prompting is a technique where you have the model generate knowledge or context first, and then feed that context back in a subsequent prompt. This helps separate factual recall/generation from the final answer construction.

#### Example Flow

1. Ask the model to generate background knowledge on a topic.  
2. Provide that knowledge as context to a second prompt.

In [ ]:
# Step 1: Generate knowledge
messages_knowledge = [
    {
        "role": "system",
        "content": "You are a research assistant who provides concise background information."
    },
    {
        "role": "user",
        "content": "Explain the concept of dynamic programming in simple terms."
    }
]

background_info = generate_response(messages_knowledge, temperature=0.7)
print("Background Info:\n", background_info)

# Step 2: Use that knowledge in a new query
messages_final = [
    {
        "role": "system",
        "content": (
                "You are a teaching assistant. Use the background information provided by the user "
                "to craft your final answer."
        )
    },
    {
        "role": "user",
        "content": (
            f"Background info:\n{background_info}\n\n"
            "Now, can you give me a real-world example demonstrating dynamic programming?"
        )
    }
]

final_response = generate_response(messages_final, temperature=0.7)
print("\nFinal Answer (Informed by Generated Knowledge):\n", final_response)


**Exercise:**  
- Try this technique for other topics (e.g., “quantum computing,” “k-means clustering,” etc.).  
- Compare how well the second answer does when you feed it the generated knowledge vs. not providing that context.

### 6. Rephrase and Respond (RaR)

With RaR, you first ask the model to restate the user’s question in its own words (often improving clarity), then provide an answer. This can reduce ambiguity.

#### Example


In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant. First rephrase the user's question, "
            "then provide your best answer."
        )
    },
    {
        "role": "user",
        "content": (
            "What's the best approach to manage microservices in a large enterprise?"
        )
    }
]

response = generate_response(messages, temperature=0.7)
print("Rephrase and Respond:\n", response)

**Exercise:**  
- Modify the user question to be longer or more ambiguous, then see if rephrasing clarifies it.

### 7. Exercises and Experiments

Now that you’ve walked through each pattern, here are a few suggestions to deepen your understanding:

1. **Combine Patterns**  
   - Use **Generated Knowledge** to build a context or scenario, and then apply **Chain of Thought** to reason about that scenario.

2. **Experiment with Self-Consistency**  
   - Use the **Tree of Thoughts** approach to generate multiple branches, then apply the self-consistency technique to see which branch is most robust.

3. **Benchmarking**  
   - For each technique, measure response times and token usage to understand cost/performance implications.

4. **Extend the Notebook**  
   - Add additional cells where you create your own advanced prompts using any combination of the patterns above.


### Summary

In this notebook, you have:
- Explored **Advanced Prompting Patterns** like CoT, ToT, Algorithm of Thoughts, Generated Knowledge, and RaR.
- Practiced how to structure prompts and parse model responses.
- Seen how to orchestrate multiple calls for advanced techniques (like self-consistency voting or multi-step knowledge generation).

Feel free to continue experimenting with these patterns to see how they can be combined or adapted to fit real-world development scenarios. You now have hands-on experience building and testing advanced prompt designs for large language models!